In [1]:
!pip install rdflib
!pip install opendatasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import FOAF, SKOS, RDF, RDFS, XSD, OWL
import opendatasets as od
import pandas as pd

## Download the datasets from kaggle

In [3]:
od.download('https://www.kaggle.com/datasets/nobelfoundation/nobel-laureates')
# TODO download the other ones

Skipping, found downloaded files in "./nobel-laureates" (use force=True to force download)


## Read the csv file

In [4]:
nobels = pd.read_csv('nobel-laureates/archive.csv')
nobels

,Year,Category,Prize,Motivation,Prize Share,Laureate ID,Laureate Type,Full Name,Birth Date,Birth City,Birth Country,Sex,Organization Name,Organization City,Organization Country,Death Date,Death City,Death Country
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,Netherlands,Male,Berlin University,Berlin,Germany,1911-03-01,Berlin,Germany
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,France,Male,NaN,NaN,NaN,1907-09-07,Châtenay,France
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),Prussia (Poland),Male,Marburg University,Marburg,Germany,1917-03-31,Marburg,Germany
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,Switzerland,Male,NaN,NaN,NaN,1910-10-30,Heiden,Switzerland
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,France,Male,NaN,NaN,NaN,1912-06-12,Paris,France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,2016,Medicine,The Nobel Prize in Physiology or Medicine 2016,"""for his discoveries of mechanisms for autophagy""",1/1,927,Individual,Yoshinori Ohsumi,1945-02-09,Fukuoka,Japan,Male,Tokyo Institute of Technology,Tokyo,Japan,NaN,NaN,NaN
965,2016,Peace,The Nobel Peace Prize 2016,"""for his resolute efforts to bring the country...",1/1,934,Individual,Juan Manuel Santos,1951-08-10,Bogotá,Colombia,Male,NaN,NaN,NaN,NaN,NaN,NaN
966,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/2,928,Individual,David J. Thouless,1934-09-21,Bearsden,United Kingdom,Male,University of Washington,"Seattle, WA",United States of America,NaN,NaN,NaN
967,2016,Physics,The Nobel Prize in Physics 2016,"""for theoretical discoveries of topological ph...",1/4,929,Individual,F. Duncan M. Haldane,1951-09-14,London,United Kingdom,Male,Princeton University,"Princeton, NJ",United States of America,NaN,NaN,NaN


## Creation of namespaces

In [5]:
graph = Graph()

NOBEL = Namespace('http://www.semanticweb.org/a3d/ontologies/2024/10/nobelOntology/')
COUNTRY = Namespace('http://eulersharp.sourceforge.net/2003/03swap/countries/')

# binding prefixes to URIs
graph.bind('nobel', NOBEL)
graph.bind('countries', COUNTRY)

graph.bind('foaf', FOAF)
graph.bind('owl', OWL)
graph.bind('rdf', RDF)
graph.bind('rdfs', RDFS)
graph.bind('skos', SKOS)
graph.bind('xsd', XSD)

## Populate the graph

In [23]:
for index, row in nobels.iterrows():
    nobel = URIRef(NOBEL[row['Category'] + str(row['Year'])]) # the URI will be nobelNamespace + Category + Year
    graph.add((nobel, RDF.type, NOBEL.NobelPrize))
    graph.add((nobel, NOBEL.year, Literal(row['Year'], datatype=XSD.gYear)))
    graph.add((nobel, NOBEL.nobelCategory, Literal(row['Category'], datatype=XSD.string)))

    # handle Prize Share
    prizeShare = str(row['Prize Share']).split('/')
    graph.add((nobel, NOBEL.prizeShare, Literal(prizeShare[1], datatype=XSD.integer)))

    # handle Motivation
    if (pd.notna(row['Motivation'])):
        # TODO topic analysis
        graph.add((nobel, NOBEL.motivationTopics, Literal(row['Motivation'], datatype=XSD.string)))

## Serialization

In [20]:
with open('test.ttl', 'w') as out:
    out.write(graph.serialize(format='turtle'))